In [2]:
import os
import pandas as pd
import numpy as np
import glob

# splitting giant bed file into 1/10 the size on a per chromosome basis

In [129]:
# root='hg38_qc4_gc50perc_tss50perc_MASTER_tfit_dreg'
root='mm10_qc4_gc50perc_tss50perc_MASTER_tfit_dreg'

df = pd.read_csv('/scratch/Shares/dowell/dbnascent/out/meta_analysis/mumerge/bidirectionals_dreg_tfit/'+root+'.bed',
                sep='\t', header=None)

In [130]:
df.columns=['chr','start','stop','merge']
# df = df[['chr', 'start', 'stop']]
df['count'] = (np.arange(len(df)))
df['count'] = (df['count']+1).apply(str)
df['region_name'] = df['chr'] + ';region_' + df['count']
df.drop(['count'], axis=1, inplace=True)

df.to_csv('/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/annotations/'+root
                              +'.bed',
                             sep='\t', header=False, index=False)

df

,chr,start,stop,merge,region_name
0,chr1,35,417,dreg,chr1;region_1
1,chr1,1160,2910,dreg,chr1;region_2
2,chr1,3701,4600,dreg,chr1;region_3
3,chr1,6402,6700,dreg,chr1;region_4
4,chr1,8718,9112,dreg,chr1;region_5
...,...,...,...,...,...
563103,chrY,90837270,90837490,dreg,chrY;region_563104
563104,chrY,90839528,90839984,dreg,chrY;region_563105
563105,chrY,90841564,90841930,dreg,chrY;region_563106
563106,chrY,90843228,90844160,dreg,chrY;region_563107


In [124]:
dd={}
for chrm in list(set(df['chr'])):
    chrdf = df[df['chr']==chrm]
    chrdf=chrdf.reset_index()

    N=10
    chrcounts=round(len(chrdf)/N)
    S = chrcounts
    frames = [chrdf.iloc[i*S:(i+1)*S].copy() for i in range(N+1) ]
    ll=[]
    for i in range(len(frames)):
        ll.append(len(frames[i]))
    print(sum(ll) == len(chrdf))
    dd[chrm]=frames

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [125]:
# for chrm in list(set(df[0])):
dcat = {}
for i in range(11):
    sdf = pd.DataFrame()
    for chrm in list(dd.keys()):
        sdf=pd.concat([sdf,dd[chrm][i]])
    dcat[i]=sdf

In [126]:
ll=[]
for i in list(dcat.keys()):
    ll.append(len(dcat[i]))
sum(ll)==len(df)

True

In [127]:
for i in range(9):
    dcat[i][['chr','start','stop','region_name']].to_csv('/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/annotations/'+root+'_'
                              +str(i)+'.bed',
                             sep='\t', header=False, index=False)
dcatlast=pd.concat([dcat[9],dcat[10]])
dcatlast[['chr','start','stop','region_name']].to_csv('/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/annotations/'+root+'_'
                              +str(9)+'.bed',
                             sep='\t', header=False, index=False)    
    

In [132]:
bedlist=glob.glob('/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/annotations/mm*')
bedlist
for bed in bedlist:
    os.system('sort -k1,1 -k2,2n ' + bed + ' > ' + bed.replace('.bed', '.sorted.bed'))  

In [134]:
1126216/2 == len(df)

True

In [4]:
# script='/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/run_rbg_hoco_mouse.sbatch'
# for i in range(10):
#     print('sbatch ' + script + ' ' + str(i))

# combining distance tables

In [4]:
num=0
tf_paths= glob.glob('/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/mouse_hocomoco_full/split_'+str(num)+
            '/distances/experimental/*')
tfs=[]
for tf in tf_paths:
    tf=tf.split('/')[-1]
    tf=tf.replace('_distances.txt','')
    tfs.append(tf) 

outdir='/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/mouse_hocomoco_full/distances/'

In [5]:
for tf in tfs:
    df=pd.DataFrame()
    for num in range(10):
        splitdf=pd.read_csv('/scratch/Shares/dowell/tajo/bidirectionals_dreg_tfit_rbg/mouse_hocomoco_full/split_'
                            +str(num)+
                            '/distances/experimental/'+tf+'_distances.txt', sep='\t')
        df=pd.concat([df,splitdf])
    df.to_csv(outdir+tf+'_distances.txt', sep='\t', index=False)